In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

import pickle

import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.preprocessing import LabelBinarizer

df_path = '../../main_dataset/'
trained_model_and_fitted_encoder_path = '../models/'
plots_path = '../plots'

# Loading model
reconstructed_model = keras.models.load_model(trained_model_and_fitted_encoder_path + \
                                              'word2vec_fb_pretrained_model.h5')

# Loading fitted encoder classes
encoder = LabelBinarizer()
encoder.classes_ = np.load(trained_model_and_fitted_encoder_path + 'type_texts.npy')

# Loading word index
with open('../dataset/word_index.pickle', 'rb') as handle:
    word_index = pickle.load(handle)

# Predicting

In [10]:
# Sample texts to classify intent
to_pred = ['Эмийн найдвартай байдлыг хангаж өгнө үү . Эм чимээгүй дайн үүсгэж байна.Ямарч баталгаа алга жнь: хугацаа нь дууссан гэх мэт',
           'Сайн байна уу? Та бүхэндээ баярлалаа. Миний санал: өмнөх жилийн удаан буудайн борлуулалтын үнэ 350000 төгрөг байсан. Энэ жил яагаад бууруулахгүй гэчихээд бага үнээр авав?',
           'Засгийн газраас хэрэглүүлж байгаа 100000 орон сууцны хөрөнгөө төрийн банкинд өгөөч ээ байрандаа ормоор байна',
           'Энэ засгийн газар хүн бүрд сая төгрөг өгөх болох уу?',
           'Ордноос шагнал авч байгаа хүмүүс их байна. Засгийн газрын баруун талд машин ачаад явж байна. Жилдээ нэг төрийн шагнал авч байгаа иргэдийг ингэж болдог юм уу.',
           'Эрүүл мэндийн даатгалаар Хөнгөлөлттэй эмийг эмийн сангууд сар бүрийн хэдний өдрөөс хэдний өдөр хүртэл олгодог талаар мэдээлэл өгнө үү? Мөн өрхийн эмнэлгүүд хэдэнд эмийн жор бичиж өгдөг вэ? Хөнгөлөлттэй эмийн талаар тодорхой мэдээлэл өгнө үү.',
           'БЗД ийн р хороо хогоо ачихгүй сар болж байна. Энэ тал дээр анхаарна уу.',
           'ХУД ийн р хорооны нийгмийн ажилтанд . р сард одонгийн материал бүрдүүлэх гэж хандтал хүүхэд ой хүрээгүй мөн материал бүрдүүлэлт авах болоогүй хүүхэд ой хүрсэн хойно авна гэсэн тайлбар хэлж буцаасан. .. нд Хүүхэд ой хүрсэн тул дахин асуудлаар хандтал материал бүрдүүлэлт дууссан шалтгаанаар хүлээж авсангүй. Уг асуудлаар холбогдох бүхий л шатны байгууллагад хандтал материал бүрдүүлэлт дууссан тул ямар ч боломжгүй гэж байна. Уг асуудлыг шийдвэрлэж иргэн намайг хохиролгүй болгож одонгийн материалыг минь хүлээж авч шийдвэрлэнэ үү.',
           'СБД ийн р хороо р байрны урд сагсны талбай дээр р сургууль биеийн тамирын хичээл орж шуугиан их гаргаж байна. Үүнээс болж оршин суугчдын амгалан тайван байдал алдагдуулж байна. Мөн сургуулийн сурагчид орцон дотор тамхи татаж байна. Арга хэмжээ авч өгнө үү.',
           'Хөвсгөл аймгийн Түнэл сумаас: Сэтгэцийн эрүүл мэндийн р тасагт хоног хэвтэж хүүхдээ эмчлүүлээд гарсан. Картан дээр нь ямар эмчилгээ хийлгэсэн талаар бичсэнгүй. Мөн ямарч зөвөлгөө өгсөнгүй. Эмчилгээ хийсэн эмч нь Насанжаргал. Одоо холбогдох гэхээр утсаа авахгүй байна. Ямар эмчилгээ хийлгэсэн болон одоо яах талаараа зөвөлгөө авахыг хүсч байна.'
          ]

In [11]:
import sentencepiece as spm
import pandas as pd
import numpy as np
import time

from sklearn.feature_extraction.text import *
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

sp = spm.SentencePieceProcessor()
sp.Load('../mongolian_bert_sentencepiece/mn_uncased.model')

def sp_tokenize(w):
    return sp.EncodeAsPieces(w)

In [12]:
import string
import nltk

stopwordsmn = ['аа','аанхаа','алив','ба','байдаг','байжээ','байна','байсаар','байсан',
               'байхаа','бас','бишүү','бол','болжээ','болно','болоо','бэ','вэ','гэж','гэжээ',
               'гэлтгүй','гэсэн','гэтэл','за','л','мөн','нь','тэр','уу','харин','хэн','ч',
               'энэ','ээ','юм','үү','?','', '.', ',', '-','ийн','ын','тай','г','ийг','д','н',
               'ний','дээр','юу']

to_pred_preprocessed = []

for entry in to_pred:
    sentences = nltk.sent_tokenize(entry)
    content_sentences_stopwords = []
    
    for sentence in sentences:
        tokens = nltk.word_tokenize(sentence)
        tokens = [w.lower() for w in tokens]
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in tokens]
        words = [word for word in stripped if word.isalpha()]
        words_stopwords = [w for w in words if not w in stopwordsmn]
        
        content_sentences_stopwords.append(words_stopwords)
            
    to_pred_preprocessed.append(content_sentences_stopwords)

In [13]:
import itertools

MAX_LEN = 512

def encode_content(text):
    return [word_index.get(i, 2) for i in text]

for i in range(len(to_pred_preprocessed)):
    to_pred_preprocessed[i] = list(itertools.chain(*to_pred_preprocessed[i]))[:MAX_LEN]


to_pred_entry = [encode_content(sent) for sent in to_pred_preprocessed]
to_pred_entry = keras.preprocessing.sequence.pad_sequences(to_pred_entry,
                                                           value = word_index["<PAD>"],
                                                           padding = 'post',
                                                           maxlen = MAX_LEN)

In [14]:
print('Text words:', to_pred_preprocessed[0], '\n')
print('Encoded shape:', to_pred_entry[7].shape, '\n')
print('Words encoded and padded:', to_pred_entry[7])

Text words: ['эмийн', 'найдвартай', 'байдлыг', 'хангаж', 'өгнө', 'эм', 'чимээгүй', 'дайн', 'үүсгэж', 'байнаямарч', 'баталгаа', 'алга', 'жнь', 'хугацаа', 'дууссан', 'гэх', 'мэт'] 

Encoded shape: (512,) 

Words encoded and padded: [  274  4778   582   389  8841  4778  1018  1960  4808  5298 53615   486
   511  1015  4808 11691   541  4225   486   511  3809    63  1363  5901
  2981 20069  1699   486   511  3809   539   837   925 53615  4808 11691
   140  7790   325  4362   363   925   926  2567  3351  1538 53615  4808
 11691   140   539   317  4303   363   189   641   105  3456 21310  1021
  1960  5218  2168   325   295  8437     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0    

In [15]:
for i in range(len(to_pred_entry)):
    data_words = " ".join(to_pred_preprocessed[i])
    data_indexes = to_pred_entry[i]
    print(f'Text_{i}:', data_words)

    pred = reconstructed_model.predict([[data_indexes]])
    max_arr = pred[np.argmax(pred)]
    ind = np.where(max_arr == max_arr.max())
    print('Predicted type:', encoder.classes_[ind][0], '\n')

Text_0: эмийн найдвартай байдлыг хангаж өгнө эм чимээгүй дайн үүсгэж байнаямарч баталгаа алга жнь хугацаа дууссан гэх мэт
Predicted type: Гомдол 

Text_1: сайн та бүхэндээ баярлалаа миний санал өмнөх жилийн удаан буудайн борлуулалтын үнэ төгрөг жил яагаад бууруулахгүй гэчихээд бага үнээр авав
Predicted type: Гомдол 

Text_2: засгийн газраас хэрэглүүлж байгаа орон сууцны хөрөнгөө төрийн банкинд өгөөч байрандаа ормоор
Predicted type: Гомдол 

Text_3: засгийн газар хүн бүрд сая төгрөг өгөх болох
Predicted type: Гомдол 

Text_4: ордноос шагнал авч байгаа хүмүүс их засгийн газрын баруун талд машин ачаад явж жилдээ нэг төрийн шагнал авч байгаа иргэдийг ингэж болдог
Predicted type: Гомдол 

Text_5: эрүүл мэндийн даатгалаар хөнгөлөлттэй эмийг эмийн сангууд сар бүрийн хэдний өдрөөс хэдний өдөр хүртэл олгодог талаар мэдээлэл өгнө өрхийн эмнэлгүүд хэдэнд эмийн жор бичиж өгдөг хөнгөлөлттэй эмийн талаар тодорхой мэдээлэл өгнө
Predicted type: Талархал 

Text_6: бзд р хороо хогоо ачихгүй сар болж тал